# Playground

## Imports

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import re

## Scraping Festival lineups

In [2]:
def get_festival_names():
    
    url = 'https://www.musicfestivalwizard.com/festival-guide/uk-festivals/'

    HEADERS = {
        'User-Agent':
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'
    }

    response = requests.get(url,headers=HEADERS)

    soup = BeautifulSoup(response.content, "html.parser")
    
    page_nums = soup.find_all('a',class_='page-numbers')
    
    page_num = [int(num.text) for num in page_nums if num.text.isdigit()][-1]
    
    festival_urls = []
    
    for i in range(page_num):
        
        page_url = f'https://www.musicfestivalwizard.com/festival-guide/uk-festivals/page/{i+1}/'
        
        response = requests.get(page_url,headers=HEADERS)
        
        soup = BeautifulSoup(response.content, "html.parser")

        festivals = soup.find_all("div", class_="entry-title search-title")

        festival_names = [festival.find('h2').find('a')['href'] for festival in festivals if festival.find('h2') != None]
    
        festival_urls += [re.findall(r'/.*/(.*)/', festival)[0] for festival in festival_names]

    return festival_urls

In [3]:
festival_names = get_festival_names()

In [4]:
def get_festival_lineup(festival):

    base_url = f'https://www.musicfestivalwizard.com/festivals/{festival}/'

    HEADERS = {
        'User-Agent':
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'
    }

    response = requests.get(url=base_url, headers=HEADERS)
        
    soup = BeautifulSoup(response.content, "html.parser")
    
    if bool(soup.find_all('div', class_="lineupblock")):
    
        artists = [
            artist.text for artist in soup.find_all(
                'div', class_="lineupblock")[0].find_all('li')
        ]

        return artists
    
    else:
        pass

In [5]:
def festivals_and_artists(festival_names):
    
    artists_dict = {}
    
    for festival in festival_names:
        
        lineup = get_festival_lineup(festival)
        
        artists_dict[festival] = lineup
        
    return artists_dict

In [6]:
festival_dict = festivals_and_artists(get_festival_names())

In [7]:
festival_dict_clean = {k:v for k,v in festival_dict.items() if v}

In [10]:
# for key in festival_dict_clean.keys():
    
#     num_artists = len(festival_dict_clean[key])
    
#     print(f"{key} has {num_artists} artists")


In [19]:
festival_names = [key[:-5].replace('-',' ').title() for key in festival_dict_clean.keys()]

## Spotify API: Playlist -> data

## Getting artist details

## Model similar songs